### Import and Device Configuration

In [2]:
import torch
from copy import deepcopy
import timm
torch.set_printoptions(precision = 6, sci_mode = False)

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/Users/bigsad/opt/anaconda3/envs/PyTEI/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
sys.path.append('../../pytei')

In [10]:
timm.list_models()

['caformer_b36',
 'caformer_m36',
 'caformer_s18',
 'caformer_s36',
 'coatnet_0_224',
 'coatnet_0_rw_224',
 'coatnet_1_224',
 'coatnet_1_rw_224',
 'coatnet_2_224',
 'coatnet_2_rw_224',
 'coatnet_3_224',
 'coatnet_3_rw_224',
 'coatnet_4_224',
 'coatnet_5_224',
 'coatnet_bn_0_rw_224',
 'coatnet_nano_cc_224',
 'coatnet_nano_rw_224',
 'coatnet_pico_rw_224',
 'coatnet_rmlp_0_rw_224',
 'coatnet_rmlp_1_rw2_224',
 'coatnet_rmlp_1_rw_224',
 'coatnet_rmlp_2_rw_224',
 'coatnet_rmlp_2_rw_384',
 'coatnet_rmlp_3_rw_224',
 'coatnet_rmlp_nano_rw_224',
 'coatnext_nano_rw_224',
 'convformer_b36',
 'convformer_m36',
 'convformer_s18',
 'convformer_s36',
 'davit_base',
 'davit_base_fl',
 'davit_giant',
 'davit_huge',
 'davit_huge_fl',
 'davit_large',
 'davit_small',
 'davit_tiny',
 'dm_nfnet_f0',
 'dm_nfnet_f1',
 'dm_nfnet_f2',
 'dm_nfnet_f3',
 'dm_nfnet_f4',
 'dm_nfnet_f5',
 'dm_nfnet_f6',
 'eca_nfnet_l0',
 'eca_nfnet_l1',
 'eca_nfnet_l2',
 'eca_nfnet_l3',
 'eca_vovnet39b',
 'efficientformerv2_l',
 'effi

### Definition of Target Model

In [5]:
test_model = timm.create_model('tiny_vit_5m_224', pretrained = True, num_classes = 5)

In [ ]:
# with open("targets", "w") as f:
#     for name, param in test_model.named_parameters():
#         print(f"{name}: {param.shape}")
#         f.write(f";{name}\n")

### Error Injection using PyTEI

In [8]:
from pytei import Injector

model = deepcopy(test_model).to(device)
model.eval()
test_input = torch.normal(size = (1, 3, 224, 224), mean = 0, std = 1).to(device)

with torch.no_grad():
    injector = Injector('./targets', p = 1e-7, device = device, verbose = True, mitigation = 'clip')
    print('----------Error free----------')
    print('Outputs (error-free):', model(test_input))
    
    print('----------Error Injected----------')
    injector.inject(model)
    print('Outputs (error-injected):', model(test_input))

    print('----------Error Mitigated----------')
    model = deepcopy(test_model).to(device)
    model.eval()
    injector.inject(model, use_mitigation = True)
    print('Outputs (error-mitigated):', model(test_input))

    injector.save_error_map('../../temp/testmap.pt', sparse = True)
    injector.load_error_map('../../temp/testmap.pt', sparse = True)

Injector initialized.
Error probability: 1e-07
Data type: torch.float32
Error model: bit
----------Error free----------
Outputs (error-free): tensor([[ 0.033936,  0.075399,  0.039451, -0.013695,  0.101700]])
----------Error Injected----------
The following parameters have been injected:
dict_keys(['stages.0.blocks.0.conv1.conv.weight', 'stages.0.blocks.0.conv2.conv.weight', 'stages.0.blocks.0.conv3.conv.weight', 'stages.0.blocks.1.conv1.conv.weight', 'stages.0.blocks.1.conv2.conv.weight', 'stages.0.blocks.1.conv3.conv.weight', 'stages.1.downsample.conv1.conv.weight', 'stages.1.downsample.conv2.conv.weight', 'stages.1.downsample.conv3.conv.weight', 'stages.1.blocks.0.attn.qkv.weight', 'stages.1.blocks.0.attn.proj.weight', 'stages.1.blocks.0.mlp.fc1.weight', 'stages.1.blocks.0.mlp.fc2.weight', 'stages.1.blocks.0.local_conv.conv.weight', 'stages.1.blocks.1.attn.qkv.weight', 'stages.1.blocks.1.attn.proj.weight', 'stages.1.blocks.1.mlp.fc1.weight', 'stages.1.blocks.1.mlp.fc2.weight', 'stage